In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('max_rows', 6)  # max number of rows to show in this notebook — to save space!
import seaborn as sns  # for better style in plots

# 1D analysis: `pandas`!

For 1D analysis, we are generally thinking about data that varies in time, so time series analysis. The `pandas` package is particularly suited to deal with this type of data, particularly having very convenient methods for interpreting, searching through, and using time representations.

Let's start with the example we started the class with: taxi rides in New York City.

In [ ]:
df = pd.read_csv('../data/yellow_tripdata_2016-05-01_decimated.csv', header=0, parse_dates=[0, 2], index_col=[0])

What do all these (and other) input keyword arguments do?

* header: tells which row of the data file is the header, from which it will extract column names
* parse_dates: try to interpret the values in `[col]` or `[[col1, col2]]` as dates, to convert them into `datetime` objects.
* index_col: if no index column is given, an index counting from 0 is given to the rows. By inputting `index_col=[column integer]`, that column will be used as the index instead. This is usually done with the time information for the dataset.
* skiprows: can skip specific rows, `skiprows=[row number list]`, or number of rows to skip, `skiprows=[number of rows integer]`.


In [ ]:
df

We can check to make sure the date/time information has been read in as the index, which allows us to reference the other columns using this time information really easily:

In [ ]:
df.index

From this we see that the index is indeed using the timing information in the file, and we can see that the `dtype` is `datetime`.

We can now access the file information using keyword arguments, like so:

In [ ]:
df['trip_distance']

We can plot in this way, too:

In [ ]:
df['trip_distance'].plot(figsize=(14,6))

One of the biggest benefits of using `pandas` is being able to easily reference the data in intuitive ways. For example, because we set up the index of the dataframe to be the date, we can pull out data using dates. In the following, we pull out all data from the first hour of the day:

In [ ]:
df['2016-05-01 00']

Here we further subdivide to examine the passenger count during that time period:

In [ ]:
df['2016-05-01 00']['passenger_count']

---
###  *Exercise*

> Figure out how to access the data from dataframe `df` for the first three hours of the day at once. Plot the tip amount (`tip_amount`) for this time period.

> After you can make a line plot, try making a histogram of the data. Play around with the data range and the number of bins.

---

## `groupby`

We can add data to our dataframe very easily. Below we add an index that gives the minute in the hour throughout the day.

We now can use the values from the key `minute` to compute the average of properties over all of the hours in the file, by minute. We access the data in the dataframe, `groupby` the minute of the hour (spanning all of the hours), and then compute the mean. For this dataset, this isn't necessarily a very meaningful computation since why would one part of the hour consistently have different types of taxi rides?

Note that we can change many `plot` parameters directly from `pandas`.

In [ ]:
df.loc[:, 'minute'] = df.index.minute  # adding a field for the minute of the hour
df.groupby('minute').aggregate(np.mean)['payment_type'].plot(color='k', grid=True, figsize=(14, 4), lw=3)

## Resampling

Sometimes we want our data to be at a different sampling frequency that we have. Changing this is called resampling. We can upsample to increase the number of data point within a given dataset (or decrease the period between points) or we can downsample to decrease the number of data points.

Let's read in the wind data file that we have used before to examine this further:

In [ ]:
df2 = pd.read_table('../data/burl1h2010.txt', header=0, skiprows=[1], delim_whitespace=True, 
                    parse_dates={'dates': ['#YY', 'MM', 'DD', 'hh']},  index_col=0)
df2

In [ ]:
df2.index

The data is sampled every hour. We could downsample it to be once a day instead. A method needs to be used for how to combine the data over the downsampling period — the default is 'mean'. We could use the maximum value over the 1-day period to represent each day:

In [ ]:
df2.resample('1d', 'max')#['DEWP']  # now the data is daily

We can instead upsample our data:

In [ ]:
df2.resample('15min')

Now the dates are every 15 minutes, as per our downsampling. However, the data is just filled in as NaNs until we tell it something else to do. A reasonable option is to interpolate:

In [ ]:
df2.resample('15min').interpolate()

---
###  *Exercise*

> We looked at the trip distance earlier, but it was hard to tell what was going on with so much data. Resample this high resolution data to be lower resolution so that any trends in the information are easier to see. By what method do you want to do this downsampling? Plot your results.

---